In [ ]:
import os
import awswrangler as wr
from dotenv import load_dotenv
from s3pathlib import S3Path

if "notebook" in os.getcwd():
    os.chdir("..")

from src.notebook_util import setup_epa_s3_session

In [ ]:
epa_boto3_session = setup_epa_s3_session()

bird_object_list = wr.s3.list_objects(
    f"s3://{os.environ['epa_bucket']}/",
    boto3_session=epa_boto3_session
)

In [ ]:
def eval_object_length(s3uri, target_size, boto_session):
    result = wr.s3.describe_objects(
        s3uri,
        boto3_session=boto_session,
    )

    object_size = int(result[s3uri]["ContentLength"])

    if object_size > target_size:
        return True
    else:
        return False

In [ ]:
locations_of_interest = ["AVLF-2", "AVCR-6", "AVCR-8", "AVPP-3"]

In [ ]:
for s3_file_path in bird_object_list:
    epa_s3path = S3Path.from_s3_uri(s3_file_path)

    epa_s3path_parts = epa_s3path.parts
    epa_s3path_parent = epa_s3path.parent

    mtsu_s3path = epa_s3path.copy()
    mtsu_s3path = mtsu_s3path.change(new_bucket="mtsu-bird-sound-data-bucket")
    mtsu_s3path_parent = mtsu_s3path.parent.uri
    mtsu_s3path_parent = mtsu_s3path_parent.replace("Data", "Data2")

    if "AVLF-2" in epa_s3path_parts:
        if eval_object_length(epa_s3path.uri, target_size=50000000, boto_session=epa_boto3_session):

            print(epa_s3path.uri)
            print(epa_s3path_parent.uri)
            print(mtsu_s3path_parent)
            
        # wr.s3.copy_objects(
        #     paths=[epa_s3path.uri],
        #     source_path=epa_s3path_parent.uri,
        #     target_path=mtsu_s3path_parent.uri,
        #     boto3_session=epa_boto3_session,
        # )